In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### External Imports

In [11]:
import os
import torch
import torchvision as tv
import random
import cv2 as cv
import numpy as np
import typing as t
import pathlib as pb
from ultralytics import YOLO

### Internal Imports

In [12]:
from res import Assets
from model import FasterRCNNMOD
from alg import VehicleOccupancy, VehicleTracking
from data import CarTrafficDataset, CarTrafficTaskOneDict, CarTrafficTaskTwoDict

### Environment Setup

In [13]:
PATH_ROOT: pb.Path = pb.Path('..')
PATH_ASSETS: pb.Path = PATH_ROOT / 'assets'
PATH_MASKS: pb.Path = PATH_ASSETS / 'masks'
PATH_MODELS: pb.Path = PATH_ASSETS / 'models'
PATH_DATA: pb.Path = PATH_ROOT / 'data' / 'train'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
SEED: int = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
np.random.default_rng(SEED)
torch.set_grad_enabled(False)

### Data Loading

In [15]:
dataset: CarTrafficDataset = CarTrafficDataset(PATH_DATA)
assets: Assets = Assets(PATH_ASSETS)

### Task 1

In [16]:
# # Create the task solver
# vo = VehicleOccupancy(assets, device)

# # Solve each example in the task subset
# for i in range(len(dataset.task_1)):
#     data: CarTrafficTaskOneDict = dataset.task_1[i]
#     output: t.List[int] = vo(data)
#     output_file: pb.Path = dataset.task_1.path / f"{data['context']:02}_{data['example']}_predicted.txt"
#     cv.destroyAllWindows()

#     # Write the answers
#     with open(output_file, 'w') as out_file:
#         out_file.write(str(len(output)) + os.linesep)
#         for i, (query, answer) in enumerate(zip(data['query'], output)):
#             out_file.write(str(query) + ' ' + str(answer))
#             if i < len(output) - 1:
#                 out_file.write(os.linesep)

### Task 2

In [19]:
# Iterate through each video in the dataset
for i in range(0, len(dataset.task_2)):
    # Create task solver for each video
    context: CarTrafficTaskTwoDict = next(iter(dataset.task_2[i]))
    tracks: t.List[t.Tuple[int, int, int, int]] = []
    vt = VehicleTracking(assets, device, debug=True)

    # Track the vehicle in each frame of the video
    for data in dataset.task_2[i]:
        # Compute BBOXes
        bbox: t.Tuple[int, int, int, int] | None = vt(data)

        # If the car is outside then stop
        if bbox is None:
            break

        # Draw
        print(bbox)
        image: np.ndarray = data['image'].copy()
        image: np.ndarray = cv.rectangle(image, (bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]), color=(0, 0, 255), thickness=5)
        tracks.append(bbox)
        context = data

        # Draw
        cv.imshow('video', image)
        cv.waitKey(int(1_000. / data['fps']))

    # Write the answers
    output_file: pb.Path = dataset.task_2.path / f"{context['context']:02}_predicted.txt"
    with open(output_file, 'w') as out_file:
        out_file.write(f"{context['total']} -1 -1 -1 -1" + os.linesep)
        for j, (x, y, x_, y_) in enumerate(tracks):
            out_file.write(f'{j} {x} {y} {x_} {y_}' + os.linesep)


0: 320x640 17 cars, 3.5ms
Speed: 1.0ms preprocess, 3.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 17 cars, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.8ms
Speed: 1.0ms preprocess, 2.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


(1314, 673, 1643, 880)
(1314, 711, 1613, 879)
(1309, 703, 1608, 879)
(1308, 703, 1609, 879)
(1308, 703, 1609, 879)
(1308, 703, 1609, 879)
(1300, 700, 1596, 878)



0: 320x640 18 cars, 4.2ms
Speed: 1.3ms preprocess, 4.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 2.7ms
Speed: 1.0ms preprocess, 2.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 1 truck, 2.7ms
Speed: 1.1ms preprocess, 2.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 1 truck, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 1 truck, 2.6ms
Speed: 1.1ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 1 truck, 2.7ms
Speed: 1.2ms preprocess, 2.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


(1300, 700, 1597, 878)
(1294, 696, 1584, 878)
(1294, 696, 1584, 879)
(1288, 696, 1575, 879)
(1288, 696, 1575, 879)
(1283, 691, 1564, 878)
(1283, 691, 1564, 878)



0: 320x640 16 cars, 2.9ms
Speed: 1.0ms preprocess, 2.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 2.7ms
Speed: 1.0ms preprocess, 2.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16 cars, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 15 cars, 2.6ms
Speed: 1.1ms preprocess, 2.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 15 cars, 3.3ms
Speed: 1.0ms preprocess, 3.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 15 cars, 3.5ms
Speed: 1.0ms preprocess, 3.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


...


0: 320x640 6 cars, 1 truck, 4.5ms
Speed: 1.3ms preprocess, 4.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 6 cars, 1 truck, 4.0ms
Speed: 1.3ms preprocess, 4.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 6 cars, 1 truck, 3.9ms
Speed: 1.3ms preprocess, 3.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 6 cars, 1 truck, 3.9ms
Speed: 1.3ms preprocess, 3.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 6 cars, 1 truck, 3.9ms
Speed: 1.3ms preprocess, 3.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 7 cars, 1 truck, 3.7ms
Speed: 1.5ms preprocess, 3.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 7 cars, 1 truck, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


(416, 179, 491, 232)
(412, 177, 488, 231)
(413, 177, 488, 231)
(413, 178, 486, 229)
(413, 178, 486, 229)
(412, 175, 484, 230)
(412, 175, 483, 230)



0: 320x640 6 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 6 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 6 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 6 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 7 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 7 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 8 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


(407, 174, 479, 225)
(408, 174, 479, 225)
(408, 174, 479, 225)
(408, 174, 479, 225)
(411, 176, 475, 224)
(410, 176, 475, 224)
(400, 172, 474, 220)



0: 320x640 8 cars, 3.9ms
Speed: 1.3ms preprocess, 3.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 7 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 7 cars, 3.7ms
Speed: 1.3ms preprocess, 3.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 8 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 8 cars, 3.8ms
Speed: 1.3ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


### Task 3

In [7]:
model = YOLO(model='./private/vistraff/first-attempt3/weights/best.pt', task='track')

### Prediction

### Saving and evaluating the results

In [ ]:
d